(3) 如何选择催化剂组合与温度，使得在相同实验条件下C4烯烃收率尽可能高。若使温度低于350度，又如何选择催化剂组合与温度，使得C4烯烃收率尽可能高。

In [99]:
import pandas as pd

附件1 = pd.read_pickle('附件1.pickle')

rc = ['温度', 'Co', 'SiO2', 'HAP', '乙醇浓度']
rs = ['乙醇转化率', '乙烯选择性', 'C4烯烃选择性', '乙醛选择性', '碳数为4-12脂肪醇选择性', '甲基苯甲醛和甲基苯甲醇选择性', '其他生成物的选择性']

In [100]:
from sklearn.svm import NuSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

exp = {
    '乙醇转化率': rc,
    '乙醛选择性': rc + ['乙醇转化率'],
    '乙烯选择性': rc + ['乙醇转化率', '乙醛选择性'],
    '碳数为4-12脂肪醇选择性': rc + ['乙醇转化率', '乙醛选择性'],
    'C4烯烃选择性': rc + ['乙醇转化率', '乙醛选择性', '乙烯选择性'],
    '甲基苯甲醛和甲基苯甲醇选择性': rc + ['乙醇转化率', '乙醛选择性', '碳数为4-12脂肪醇选择性'],
    '其他生成物的选择性': rc + ['乙醇转化率', '乙烯选择性', 'C4烯烃选择性', '乙醛选择性', '碳数为4-12脂肪醇选择性', '甲基苯甲醛和甲基苯甲醇选择性']
}

model = {m: {i: make_pipeline(StandardScaler(), NuSVR()) for i in exp.keys()} for m in [0, 1]}

for m in [0, 1]:
    for i in exp.items():
        df = 附件1.loc[附件1['装料方式'] == m]
        X = df[i[1]]
        y = df[i[0]]
        model[m][i[0]].fit(X, y)
        print(model[m][i[0]].score(X, y))

0.9650552932187823
0.97878243360293
0.9986240361382679
0.9667574712816367
0.9649161022406636
0.9640063778030599
0.9957208492959726
0.9893751622401042
0.9562680493236586
0.9989323677688843
0.9971938983915566
0.9988404028769686
0.9926532000080845
0.9995044338463652


In [128]:
t = {m: {i: model[m][i].get_params() for i in rs} for m in [0, 1]}

from sko.SA import SACauchy


sa = SACauchy(func=E, x0=[673.15, 0.000001980, 0.000198020, 0.0002, 0.000000015], T_max=1e-8, T_min=1e-16, L=300, max_stay_counter=150)
best_x, best_y = sa.run()
print('best_x:', best_x, 'best_y', best_y)

In [108]:
import re

def wt(desc, mode=False):
    coc = float(re.search(r'(?<= )\d+(\.\d+)?(?=wt)', desc).group()) / 100
    cosio2 = float(re.search(r'\d+(\.\d+)?(?=mg )', desc).group()) / 10**6
    if mode:
        co = cosio2 * coc
        sio2 = cosio2 * (1 - coc)
    else:
        co = cosio2 * coc / (coc + 1)
        sio2 = cosio2 * 1 / (coc + 1)
    return co, sio2

def convert(t, s):
    Co ,SiO2 = wt(s)
    HAP = 0
    if re.search(r'无HAP', s):
        w = float(re.search(r'(?<= )\d+(\.\d+)?(?=mg石英砂)', s).group()) / 10**6
        SiO2 += w
    else:
        HAP = float(re.search(r'(?<= )\d+(\.\d+)?(?=mg HAP)', s).group()) / 10**6
    乙醇浓度 = float(re.search(r'(?<=乙醇浓度)\d+(\.\d+)?(?=ml)', s).group()) / 6 / 10**7

    return t + 273.15, Co, SiO2, HAP, 乙醇浓度

print(convert(325, '100mg 1wt%Co/SiO2- 100mg HAP-乙醇浓度0.9ml/min'))

(598.15, 9.900990099009902e-07, 9.900990099009902e-05, 0.0001, 1.5e-08)


In [135]:
def trevnoc(t, co, sio2, hap, ethanol):
    wt = (co + sio2 / sio2 - 1) * 100
    cosio2 = (co + sio2) * 1e6
    hap *= 1e6
    ethanol *= 6e7
    return t - 273.15, '{:.0f}mg {:.4f}wt%Co/SiO2- {:.0f}mg HAP-乙醇浓度{:.2f}ml/min'.format(co + cosio2, wt, hap, ethanol)

print(trevnoc(7.29150000e+02, 6.50000000e-06, 3.69009901e-04, 3.75509901e-04, 3.20000000e-08))

(456.0, '376mg 0.0007wt%Co/SiO2- 376mg HAP-乙醇浓度1.92ml/min')


In [124]:
from sko.PSO import PSO

C350 = True
LM = 0

min_x = [273.15, 0, 0, 0, 0]
max_x = [1000, 0.01, 0.99, 1, 1]

def E(x, lm=0):
    x = np.expand_dims(x, axis=0)
    y_ethanol = model[lm]['乙醇转化率'].predict(x)
    x = np.expand_dims(np.append(x, y_ethanol), axis=0)
    y_ethanal = model[lm]['乙醛选择性'].predict(x)
    x = np.expand_dims(np.append(x, y_ethanal), axis=0)
    y_ethylene = model[lm]['乙烯选择性'].predict(x)
    x = np.expand_dims(np.append(x, y_ethylene), axis=0)
    y_alkene = np.expand_dims(model[lm]['C4烯烃选择性'].predict(x), axis=0)
    t = y_alkene * y_ethanol
    return t

if not LM and C350:
    best_x = [598.15, 3.921568627450981e-06, 0.00019607843137254904, 0.0002, 2.7999999999999996e-08]
elif not LM and not C350:
    best_x = [673.15, 1.9801980198019803e-06, 0.00019801980198019803, 0.0002, 1.5e-08]
elif LM and C350:
    best_x = [598.15, 9.900990099009902e-07, 9.900990099009902e-05, 0.0001, 1.5e-08]
elif LM and not C350:
    best_x = [673.15, 9.900990099009902e-07, 9.900990099009902e-05, 0.0001, 1.5e-08]

print(E(best_x))

[[0.18852667]]


In [ ]:
if C350:
    max_x[0] = 623.15

pso = PSO(func=E, n_dim=5, pop=50, max_iter=200, lb=[273.15, 0, 0, 0, 1e-14], ub=[1000, 1e-5, 1e-3, 1e-3, 1e-7], w=0.08, c1=5, c2=0.05)
pso.run()
print('best_x is ', pso.gbest_x, 'best_y is', pso.gbest_y)

import matplotlib.pyplot as plt
import pandas as pd

plt.plot(pd.DataFrame(sa.best_y_history).cummin(axis=0))
plt.show()

"""Monte Carlo"""

epoch = 10000000
ratio = 0.2
maxy = 0
maxx = None

xmax = X.max()
xmin = X.min()

for i in tqdm(range(epoch)):
    x = [[rd(xmin[c] * (1 - ratio), xmax[c] * (1 + ratio)) for c in X.columns]]
    y = en.predict(x)
    if y > maxy:
        maxy = y
        maxx = x

In [133]:
from tqdm.notebook import trange
import numpy as np
import random

C350 = True
LM = 1

def E(x, lm=0):
    x = np.expand_dims(x, axis=0)
    y_ethanol = model[lm]['乙醇转化率'].predict(x)
    x = np.expand_dims(np.append(x, y_ethanol), axis=0)
    y_ethanal = model[lm]['乙醛选择性'].predict(x)
    x = np.expand_dims(np.append(x, y_ethanal), axis=0)
    y_ethylene = model[lm]['乙烯选择性'].predict(x)
    x = np.expand_dims(np.append(x, y_ethylene), axis=0)
    y_alkene = np.expand_dims(model[lm]['C4烯烃选择性'].predict(x), axis=0)
    t = y_alkene * y_ethanol
    return t

min_x = [273.15, 0, 0, 0, 0]
max_x = [1000, 0.01, 0.99, 1, 1]

if not LM and C350:
    best_x = [598.15, 3.921568627450981e-06, 0.00019607843137254904, 0.0002, 2.7999999999999996e-08]
elif not LM and not C350:
    best_x = [673.15, 1.9801980198019803e-06, 0.00019801980198019803, 0.0002, 1.5e-08]
elif LM and C350:
    best_x = [598.15, 9.900990099009902e-07, 9.900990099009902e-05, 0.0001, 1.5e-08]
elif LM and not C350:
    best_x = [673.15, 9.900990099009902e-07, 9.900990099009902e-05, 0.0001, 1.5e-08]

if C350:
    max_x[0] = 623.15

best_y = E(best_x)
pre_y = best_y

x = best_x[:]

lr = [1, 1e-7, 1e-5, 1e-5, 1e-9]
epoch = 10000000
cnt = 0

path_x = []
path_y = []

with trange(epoch) as tr:
    for i in tr:
        r = random.randint(0, 3)
        if r == 3:
            r += 1
        x[r] = max(x[r], min_x[r])
        x[r] = min(x[r], max_x[r])
        x[r] += (random.randint(-1, 1)) * lr[r]
        x[3] = x[1] + x[2]
        path_x.append(x)
        y = E(x)
        path_y.append(y[0][0])
        if y - pre_y < 1e-8:
            cnt += 1
        if cnt > 10000:
            break
        if y > best_y:
            best_y = y
            best_x = x
            tr.set_postfix(best_y=best_y)
        pre_y = y

path_y = np.array(path_y)
path_x = np.array(path_x)

e = path_y.argmax()

t = path_x[e]

print(e)
print(t)
print(best_y[0][0])

  0%|          | 0/10000000 [00:00<?, ?it/s]

12743
[6.11150000e+02 1.50000000e-06 3.69009901e-04 3.70509901e-04
 6.70000000e-08]
0.20918571250988757


best_x = [673.15, 0.000001980, 0.000198020, 0.0002, 0.000000015]
best_y = E(best_x)
x = best_x[:]

with trange(600, 900) as tr:
    for i in tr:
        for j in range(1, 10):
            for k in range(90, 100):
                for l in range(1, 100):
                    x[0] = i
                    x[1] = j * 0.000001
                    x[2] = k * 0.000001
                    x[3] = x[1] + x[2]
                    x[4] = l * 0.000000001
                    tr.set_postfix(co=j, sio2=k, ethanol=l, best_y=best_y)
                    y = E(x)
                    if y > best_y:
                        best_x = x
                        best_y = y
                        print(best_y)